In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier

- Concateno todos los set de vistas

In [3]:
vistas_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_3_vistas.csv'), pd.read_csv('fiuba_hasta_15_abril/fiuba_3_vistas.csv'), pd.read_csv('Fiuba desde 15 Abril/fiuba_3_vistas.csv')])
vistas_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18368813 entries, 0 to 11648229
Data columns (total 3 columns):
idAviso         int64
timestamp       object
idpostulante    object
dtypes: int64(1), object(2)
memory usage: 560.6+ MB


In [4]:
def normalize_str(date_str):
    return (date_str[:10]+' '+date_str[11:19])
    
vistas_totales['timestamp'] = vistas_totales['timestamp'].apply(normalize_str)
vistas_totales['timestamp'] = pd.to_datetime(vistas_totales['timestamp'])

vistas_totales.drop_duplicates(inplace=True)

vistas_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16323504 entries, 0 to 11648229
Data columns (total 3 columns):
idAviso         int64
timestamp       datetime64[ns]
idpostulante    object
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 498.2+ MB


In [5]:
vistas_totales.rename(columns={'idAviso':'idaviso'}, inplace=True)

In [6]:
vistas_totales.head()

,idaviso,timestamp,idpostulante
0,1111780242,2018-02-23 13:38:13,YjVJQ6Z
1,1112263876,2018-02-23 13:38:14,BmVpYoR
2,1112327963,2018-02-23 13:38:14,wVkBzZd
3,1112318643,2018-02-23 13:38:17,OqmP9pv
4,1111903673,2018-02-23 13:38:18,DrpbXDP


- Calculo cantidad de vistas de cada usuario a cada anuncio

In [7]:
vistas_totales_count = vistas_totales.groupby(by=['idaviso', 'idpostulante']).count()
vistas_totales_count.reset_index(drop=False, inplace=True)
vistas_totales_count.rename(columns={'timestamp':'veces_visto'}, inplace=True)
vistas_totales_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8597915 entries, 0 to 8597914
Data columns (total 3 columns):
idaviso         int64
idpostulante    object
veces_visto     int64
dtypes: int64(2), object(1)
memory usage: 196.8+ MB


In [8]:
vistas_totales_count.head(5)

,idaviso,idpostulante,veces_visto
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [44]:
data_set_complete = pd.read_csv('data_set_complete.csv', parse_dates=['fechapostulacion', 'ultima_vista'], dtype={'sepostulo':'int8', 'idaviso':'int64', 'idpostulante':'object', 'veces_visto':'int16', 'estudios':'category', 'estado':'category', 'edad':'int8', 'sexo':'category','nombre_zona':'category', 'tipo_de_trabajo':'category', 'nivel_laboral':'category', 'nombre_area':'category'})
data_set_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13208286 entries, 0 to 13208285
Data columns (total 16 columns):
sepostulo               int8
idaviso                 int64
idpostulante            object
fechapostulacion        datetime64[ns]
ultima_vista            datetime64[ns]
veces_visto             int16
estudios                category
estado                  category
sexo                    category
edad                    int8
titulo                  object
nombre_zona             category
tipo_de_trabajo         category
nivel_laboral           category
nombre_area             category
denominacion_empresa    object
dtypes: category(7), datetime64[ns](2), int16(1), int64(1), int8(2), object(3)
memory usage: 755.8+ MB


In [45]:
data = data_set_complete.sample(1000000)
del data_set_complete

In [46]:
data['nombre_zona'].fillna(value='Gran Buenos Aires',inplace=True)
data['tipo_de_trabajo'].fillna(value='Full-time',inplace=True)
data['nivel_laboral'].fillna(value='Senior / Semi-Senior',inplace=True)
data['nombre_area'].fillna(value='Ventas',inplace=True)
data['estado'].fillna(value='Graduado',inplace=True)
data['estudios'].fillna(value='Secundario',inplace=True)

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 1877403 to 4713976
Data columns (total 16 columns):
sepostulo               1000000 non-null int8
idaviso                 1000000 non-null int64
idpostulante            1000000 non-null object
fechapostulacion        499742 non-null datetime64[ns]
ultima_vista            134835 non-null datetime64[ns]
veces_visto             1000000 non-null int16
estudios                1000000 non-null category
estado                  1000000 non-null category
sexo                    1000000 non-null category
edad                    1000000 non-null int8
titulo                  937225 non-null object
nombre_zona             1000000 non-null category
tipo_de_trabajo         1000000 non-null category
nivel_laboral           1000000 non-null category
nombre_area             1000000 non-null category
denominacion_empresa    937172 non-null object
dtypes: category(7), datetime64[ns](2), int16(1), int64(1), int8(2), object(3)
memory usage: 

In [48]:
def df_encode_categorical_vars(df):
    for column in df.columns:
        if (df[column].dtype.name == 'category'):
            le = preprocessing.LabelEncoder()
            df[column] = le.fit_transform(df[column].astype(str))

In [49]:
df_encode_categorical_vars(data)
data.head()

,sepostulo,idaviso,idpostulante,fechapostulacion,ultima_vista,veces_visto,estudios,estado,sexo,edad,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
1877403,1,1112216893,BmZP4Rv,2018-01-15 23:39:49,NaT,1,5,2,1,24,Operarios de carga y descarga/ Villa Soldati,1,1,2,139,Grupo Cosmos
11232552,0,1112274792,akjY3z5,NaT,NaT,0,4,2,0,23,Empleado de Maestranza,5,1,4,115,Manpower
2675173,1,1112350600,LNK3e6k,2018-02-27 20:54:36,2018-02-27 20:53:05,2,6,2,1,27,Atención Telefónica - Rubro Automotriz en Mart...,5,1,4,14,Atento
3123962,1,1112453542,Nzr0zr4,2018-04-11 06:26:46,NaT,1,6,2,0,25,Administrativo /a de Facturación,1,1,4,71,Psicovision
8356717,0,1112355348,A3RbZdx,NaT,NaT,0,5,2,1,39,Operarios de Carga y Descarga - Zona San Martin,5,1,2,139,Adecco -Sales & Marketing


In [76]:
data['veces_visto'] = (pd.merge(data[['idaviso','idpostulante']], vistas_totales_count, on=['idaviso', 'idpostulante'], how='left'))['veces_visto']
data['veces_visto'].fillna(value=(0, inplace=True)
data['veces_visto'] = data['veces_visto'].astype('int16')

In [77]:
train_set, test_set, train_classes, test_classes = train_test_split(data.drop(columns=['sepostulo','idaviso','idpostulante', 'fechapostulacion','ultima_vista','titulo','denominacion_empresa']), data['sepostulo'], test_size=0.4)

In [78]:
clf = tree.DecisionTreeClassifier().fit(train_set, train_classes)
clf.score(test_set, test_classes)

0.6045325

In [34]:
test100k=pd.read_csv('test_final_100k_complete.csv', parse_dates=['fechapostulacion', 'ultima_vista'], dtype={'sepostulo':'int8', 'idaviso':'int64', 'idpostulante':'object', 'veces_visto':'int16', 'estudios':'category', 'estado':'category', 'edad':'int8', 'sexo':'category','nombre_zona':'category', 'tipo_de_trabajo':'category', 'nivel_laboral':'category', 'nombre_area':'category'})
test100k['nombre_zona'].fillna(value='Gran Buenos Aires',inplace=True)
test100k['tipo_de_trabajo'].fillna(value='Full-time',inplace=True)
test100k['nivel_laboral'].fillna(value='Senior / Semi-Senior',inplace=True)
test100k['nombre_area'].fillna(value='Ventas',inplace=True)
test100k['estado'].fillna(value='Graduado',inplace=True)
test100k['estudios'].fillna(value='Secundario',inplace=True)
df_encode_categorical_vars(test100k)
test100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
id                      100000 non-null int64
idaviso                 100000 non-null int64
idpostulante            100000 non-null object
fechapostulacion        0 non-null datetime64[ns]
ultima_vista            34161 non-null datetime64[ns]
veces_visto             100000 non-null int16
estudios                100000 non-null int64
estado                  100000 non-null int64
sexo                    100000 non-null int64
edad                    100000 non-null int8
titulo                  100000 non-null object
nombre_zona             100000 non-null int64
tipo_de_trabajo         100000 non-null int64
nivel_laboral           100000 non-null int64
nombre_area             100000 non-null int64
denominacion_empresa    99988 non-null object
dtypes: datetime64[ns](2), int16(1), int64(9), int8(1), object(3)
memory usage: 11.0+ MB


In [35]:
test100k['sepostulo'] = clf.predict(test100k.drop(columns=['id', 'idaviso','idpostulante', 'fechapostulacion','ultima_vista','titulo','denominacion_empresa']))
test100k[['id','sepostulo']].head()

,id,sepostulo
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1


In [36]:
test100k['sepostulo'].mean()

0.60417

In [37]:
test100k[['id','sepostulo']].to_csv('Resultados/tree_confillna_vecesvistosinfillna_confillnaen100k.csv', index=False)

In [79]:
gradient_boosting_clf = GradientBoostingClassifier().fit(train_set, train_classes)

In [80]:
gradient_boosting_clf.score(test_set, test_classes), gradient_boosting_clf.score(train_set, train_classes)

(0.5807425, 0.5810433333333334)